### **Homework: Open-Source LLMs**

**Q1: Running Ollama with Docker**

Run ollama with Docker. What's the version of the ollama client?

*0.2.5*

**In the terminal:**
- `docker run -it \
    --rm \
    -v ollama:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollam`
- `docker exec -it ollama bash`
- `ollama -v`
- OR `docker exec -it ollama ollama -v`

**Q2: Downloading a LLM**

Download a smaller LLM - `gemma:2b`. We're interested in the model's metadata. What's the content of the file related to `gemma`?

*{"schemaVersion":2,"mediaType":"application/vnd.docker.distribution.manifest.v2+json","config":{"mediaType":"application/vnd.docker.container.image.v1+json","digest":"sha256:887433b89a901c156f7e6944442f3c9e57f3c55d6ed52042cbb7303aea994290","size":483},"layers":[{"mediaType":"application/vnd.ollama.image.model","digest":"sha256:c1864a5eb19305c40519da12cc543519e48a0697ecd30e15d5ac228644957d12","size":1678447520},{"mediaType":"application/vnd.ollama.image.license","digest":"sha256:097a36493f718248845233af1d3fefe7a303f864fae13bc31a3a9704229378ca","size":8433},{"mediaType":"application/vnd.ollama.image.template","digest":"sha256:109037bec39c0becc8221222ae23557559bc594290945a2c4221ab4f303b8871","size":136},{"mediaType":"application/vnd.ollama.image.params","digest":"sha256:22a838ceb7fb22755a3b0ae9b4eadde629d19be1f651f73efb8c6b4e2cd0eea0","size":84}]}root@93f3bb0ad1c0:~/.ollama/models/manifests/registry.ollama.ai/library/gemma#*

In [1]:
!ollama pull gemma:2b

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling c1864a5eb193...   0% ▕                ▏    0 B/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   0% ▕                ▏    0 B/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   0% ▕                ▏    0 B/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   0% ▕                ▏    0 B/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   0% ▕                ▏    0 B/1.7 GB                  pulling manifest 
pul

**In the terminal:**
- `docker exec -it ollama bash`
- `cd cd root/.ollama/models/manifests/registry.ollama.ai/library/gemma`
- `cat 2b`

**Q3: Running the LLM**

Test the prompt: "10 * 10". What's the answer?

*The answer is 100, which is less than 10*

**In the terminal:**
- `docker exec -it ollama ollama run gemma:2b`

**Q4: Downloading the Weights**

We don't want to have to pull the weights every time we run a docker container. Let's do it once and have them available every time we start a container. What's the size of the `ollama_files/models` folder?

*How do I copy files from a different folder into docker container's working directory?*

In [12]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
                        # search only the data engineering course documents
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    doc = hit['_source']
    print(f"Course: {doc['course']}")
    print(f"Question: {doc['question']}")
    print(f"Answer: {doc['text'][:100]}...\n")

Course: machine-learning-zoomcamp
Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a ba...

Course: machine-learning-zoomcamp
Question: How do I copy files from my local machine to docker container?
Answer: You can copy files from your local machine into a Docker container using the docker cp command. Here...

Course: machine-learning-zoomcamp
Question: How do I copy files from a different folder into docker container’s working directory?
Answer: You can copy files from your local machine into a Docker container using the docker cp command. Here...



**Q5: Building a Prompt**

Use the records returned from ElasticSearch in the previous question as the template to build the context. Separate context entries by a double-linebreak (`\n\n`). Then use that context along with the query from question 3 to construct a prompt. What's the length of that prompt?

*1,462 characters*

In [13]:
context_template = """
Q: {question}
A: {text}
""".strip()

context_docs = [hit['_source'] for hit in response['hits']['hits']]
context_result = ""

for doc in context_docs:
    doc_str = context_template.format(**doc)
    context_result += ("\n\n" + doc_str)

context = context_result.strip()

prompt = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {user_question}

CONTEXT:
{context}
""".strip()

print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [14]:
print(len(prompt))

1462


**Q6: Tokens**

OpenAI charges are based on the total number of tokens in the prompt and the response. The OpenAI python package uses `tiktoken` for tokenisation. How many tokens does the prompt in the previous question have?

*322 tokens*

In [19]:
import tiktoken

# get the tokeniser for the gpt-4o model
enc = tiktoken.encoding_for_model("gpt-4o")
# encode the prompt
encoded_prompt = enc.encode(prompt)
print(encoded_prompt)
print("\n", len(encoded_prompt))


[63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122, 402, 290, 31810, 8099, 591, 290, 40251, 7862, 558, 8470, 1606, 290, 19719, 591, 290, 31810, 8099, 1261, 55959, 290, 150339, 364, 107036, 25, 3253, 621, 357, 15792, 261, 6348, 306, 261, 6788, 62275, 9282, 1715, 10637, 50738, 734, 48, 25, 3253, 621, 357, 15199, 261, 62275, 9282, 3901, 32, 25, 41281, 290, 9282, 3621, 306, 25383, 6766, 326, 151187, 290, 7251, 4859, 11, 813, 484, 480, 13217, 261, 38615, 6348, 558, 68923, 2461, 533, 278, 2230, 7962, 4859, 38615, 464, 3365, 523, 3335, 290, 9282, 382, 4279, 6788, 11, 15792, 261, 6348, 306, 290, 4857, 9282, 734, 68923, 10942, 350, 6555, 290, 9282, 26240, 446, 68923, 25398, 533, 278, 464, 6896, 26240, 29, 38615, 198, 6103, 277, 10732, 391, 79771, 1029, 48, 25, 3253, 621, 357, 5150, 6291, 591, 922, 2698, 7342, 316, 62275, 9282, 3901, 32, 25, 1608, 665, 5150, 6291, 591, 634, 2698, 7342, 1511, 261, 91238, 9282, 2360, 290, 62275, 27776, 6348, 13, 44257, 1495, 316, 621, 480, 734, 1385, 51

*Bonus: Generating the Answer*

Send the prompt to Cohere. What's the response?

In [16]:
from cohere.client import Client

co_client = Client()

response = co_client.chat(
    message=prompt,
    model="command-r",
    preamble=""
    )
print(response.text)

To execute a command in a running docker container, first find the container ID by running the command docker ps. Then, run the command docker exec -it <container-id> bash to execute a bash command. Alternatively, you can also launch the container in interactive mode to execute commands.
